## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-23-22-53-14 +0000,nyt,What’s It Like to Climb a Skyscraper?,https://www.nytimes.com/2026/01/23/world/asia/...
1,2026-01-23-22-52-00 +0000,wsj,Enphase Energy to Cut Jobs as End of Tax Credi...,https://www.wsj.com/business/enphase-energy-to...
2,2026-01-23-22-49-02 +0000,wapo,Five arrested after shooting of Indiana judge ...,https://www.washingtonpost.com/nation/2026/01/...
3,2026-01-23-22-46-00 +0000,wsj,Trump Comments on Afghan-War Veterans Spark Ba...,https://www.wsj.com/world/europe/trump-comment...
4,2026-01-23-22-43-54 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/23/weathe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2456/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,69
44,ice,22
127,new,19
169,greenland,18
245,tiktok,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2026-01-23-20-23-54 +0000,nypost,Former Eric Adams deputy Kaz Daughtry nabs job...,https://nypost.com/2026/01/23/us-news/eric-ada...,126
296,2026-01-23-02-49-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...,121
163,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...,117
281,2026-01-23-03-44-07 +0000,nypost,Trump withdraws Canadian PM Mark Carney’s Boar...,https://nypost.com/2026/01/22/us-news/trump-wi...,116
147,2026-01-23-17-00-09 +0000,latimes,Trump lawyers urge Supreme Court to block Cali...,https://www.latimes.com/politics/story/2026-01...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,126,2026-01-23-20-23-54 +0000,nypost,Former Eric Adams deputy Kaz Daughtry nabs job...,https://nypost.com/2026/01/23/us-news/eric-ada...
85,56,2026-01-23-19-52-49 +0000,nypost,Alleged psycho surgeon Michael McKee gives sto...,https://nypost.com/2026/01/23/us-news/alleged-...
296,52,2026-01-23-02-49-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...
163,48,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...
12,46,2026-01-23-22-29-26 +0000,nyt,"Ryan Wedding, Canadian Ex-Snowboarder Accused ...",https://www.nytimes.com/2026/01/23/world/canad...
234,42,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...
303,40,2026-01-23-02-20-00 +0000,wsj,California Gov. Gavin Newsom spent his days at...,https://www.wsj.com/politics/policy/the-newsom...
275,40,2026-01-23-05-17-48 +0000,bbc,TikTok closes deal to split US app from global...,https://www.bbc.com/news/articles/c3edd1l328lo...
11,36,2026-01-23-22-29-32 +0000,nyt,Hundreds of Minnesota Businesses on Strike in ...,https://www.nytimes.com/2026/01/23/us/minnesot...
292,36,2026-01-23-03-00-00 +0000,wsj,When asked what connects his two spheres of in...,https://www.wsj.com/business/media/dana-whites...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
